# AutoEncoder Wrapper Demo - Comprehensive Testing

This notebook demonstrates the complete autoencoder wrapper functionality, including:

1. **Environment Setup** - Import testing and system verification
2. **Dataset Generation** - Creating test datasets with proper formatting
3. **Model Architecture Testing** - Verifying all model types work correctly  
4. **Data Pipeline Debugging** - Ensuring data shapes and formats are correct
5. **Training Pipeline** - Running complete experiments with proper error handling
6. **Results Analysis** - Visualization and performance evaluation
7. **Wrapper Integration** - Testing the full ExperimentRunner workflow

## 🎯 Goal: Create a robust, debugged workflow for autoencoder experimentation

In [1]:
# 1. ENVIRONMENT SETUP AND IMPORT TESTING
print("=" * 60)
print("AUTOENCODER WRAPPER DEMO - COMPREHENSIVE TESTING")
print("=" * 60)
print("\\n1. Testing Environment Setup...")

import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Test core imports
try:
    import torch
    import torchvision
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    from torch.utils.data import DataLoader, TensorDataset
    print("✅ Core libraries imported successfully")
    print(f"   PyTorch: {torch.__version__}")
    print(f"   Device available: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")
except ImportError as e:
    print(f"❌ Core library import failed: {e}")
    sys.exit(1)

# Test autoencoder_lib imports
try:
    from autoencoder_lib.data.generators import generate_dataset, get_available_dataset_types
    from autoencoder_lib.models.factory import create_autoencoder, get_available_architectures
    from autoencoder_lib.experiment.runner import ExperimentRunner
    from autoencoder_lib.visualization.latent_viz import visualize_latent_space_2d
    from autoencoder_lib.visualization.reconstruction_viz import visualize_reconstructions
    print("✅ AutoEncoder library imported successfully")
except ImportError as e:
    print(f"❌ AutoEncoder library import failed: {e}")
    print("   Make sure autoencoder_lib is properly installed")
    sys.exit(1)

# Display available configurations
print("\\n📊 Available Configurations:")
try:
    architectures = get_available_architectures()
    print(f"   Architectures: {architectures}")
except:
    print("   Architectures: Could not retrieve")

try:
    dataset_types = get_available_dataset_types()
    print(f"   Dataset types: {dataset_types}")
except:
    print("   Dataset types: Could not retrieve")

print("\\n✅ Environment setup complete!")
print("=" * 60)

AUTOENCODER WRAPPER DEMO - COMPREHENSIVE TESTING
\n1. Testing Environment Setup...
✅ Core libraries imported successfully
   PyTorch: 2.7.0+cpu
   Device available: cpu
❌ AutoEncoder library import failed: No module named 'autoencoder_lib.data.generators'
   Make sure autoencoder_lib is properly installed


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
# 2. DATASET GENERATION WITH PROPER ERROR HANDLING
print("\\n2. Dataset Generation and Validation...")
print("-" * 40)

# Clean up any existing test datasets
test_dataset_dir = "comprehensive_wrapper_test"
if os.path.exists(test_dataset_dir):
    import shutil
    shutil.rmtree(test_dataset_dir)
    print(f"   Cleaned up existing {test_dataset_dir}")

# Generate a small but comprehensive test dataset
try:
    dataset_info = generate_dataset(
        dataset_type='layered_geological',
        output_dir=test_dataset_dir,
        num_samples_per_class=8,  # Small but sufficient for testing
        image_size=(32, 32),
        num_classes=3,  # Keep it simple: 3 classes
        random_seed=42,
        save_examples=True
    )
    
    print("✅ Dataset generated successfully!")
    print(f"   Directory: {test_dataset_dir}")
    print(f"   Classes: {dataset_info['label_names']}")
    print(f"   Image size: {dataset_info['image_size']}")
    print(f"   Total samples: {dataset_info['total_samples']}")
    print(f"   Samples per class: {dataset_info['samples_per_class']}")
    
    # Verify files were created
    total_files = 0
    for class_name in dataset_info['label_names']:
        class_dir = Path(test_dataset_dir) / class_name
        if class_dir.exists():
            files = list(class_dir.glob("*.png"))
            print(f"   {class_name}: {len(files)} files")
            total_files += len(files)
        else:
            print(f"   ❌ {class_name}: directory not found")
    
    print(f"   Total files created: {total_files}")
    
except Exception as e:
    print(f"❌ Dataset generation failed: {e}")
    import traceback
    traceback.print_exc()
    raise

print("\\n✅ Dataset generation complete!")
print("-" * 40)

In [ ]:
# 3. MODEL ARCHITECTURE TESTING AND VALIDATION
print("\\n3. Model Architecture Testing...")
print("-" * 40)

# Test different model architectures with proper parameters
image_size = dataset_info['image_size']
input_size = image_size[0] * image_size[1]  # 32 * 32 = 1024
input_channels = 1  # Grayscale
latent_dim = 16  # Reasonable latent dimension

models_to_test = [
    ('simple_linear', {'input_size': input_size, 'latent_dim': latent_dim}),
    ('deeper_linear', {'input_size': input_size, 'latent_dim': latent_dim}),
    ('convolutional', {'input_channels': input_channels, 'latent_dim': latent_dim}),
    ('deeper_convolutional', {'input_channels': input_channels, 'latent_dim': latent_dim})
]

created_models = {}

for arch_name, params in models_to_test:
    try:
        print(f"\\n   Testing {arch_name}...")
        print(f"   Parameters: {params}")
        
        model = create_autoencoder(architecture_name=arch_name, **params)
        
        # Test model with dummy input
        if 'input_size' in params:
            # Linear models expect flattened input
            dummy_input = torch.randn(1, input_size)
            expected_shape = (1, input_size)
        else:
            # Convolutional models expect image input
            dummy_input = torch.randn(1, input_channels, *image_size)
            expected_shape = (1, input_channels, *image_size)
        
        print(f"   Input shape: {dummy_input.shape}")
        
        # Test forward pass
        model.eval()
        with torch.no_grad():
            encoded, decoded = model(dummy_input)
        
        print(f"   ✅ {arch_name} created successfully!")
        print(f"      Encoded shape: {encoded.shape}")
        print(f"      Decoded shape: {decoded.shape}")
        print(f"      Expected shape: {expected_shape}")
        
        # Verify output shape matches input
        if decoded.shape == dummy_input.shape:
            print(f"      ✅ Output shape matches input!")
            created_models[arch_name] = {'model': model, 'params': params}
        else:
            print(f"      ❌ Shape mismatch! Expected {dummy_input.shape}, got {decoded.shape}")
            
    except Exception as e:
        print(f"   ❌ {arch_name} failed: {e}")
        import traceback
        traceback.print_exc()

print(f"\\n✅ Model testing complete! Successfully created {len(created_models)} models")
print(f"   Working architectures: {list(created_models.keys())}")
print("-" * 40)

In [ ]:
# 4. DATA LOADING AND PREPROCESSING PIPELINE
print("\\n4. Data Loading and Preprocessing...")
print("-" * 40)

def load_dataset_with_proper_shapes(dataset_dir, class_names, image_size):
    """
    Load dataset ensuring proper shapes for both linear and convolutional models.
    Returns data in both formats.
    """
    all_data = []
    all_labels = []
    
    print(f"   Loading from: {dataset_dir}")
    print(f"   Expected classes: {class_names}")
    
    for class_idx, class_name in enumerate(class_names):
        class_dir = Path(dataset_dir) / class_name
        print(f"   Processing class {class_idx}: {class_name}")
        
        if not class_dir.exists():
            print(f"      ❌ Directory not found: {class_dir}")
            continue
            
        image_files = list(class_dir.glob("*.png"))
        print(f"      Found {len(image_files)} images")
        
        for img_file in image_files:
            try:
                # Load and preprocess image
                img = Image.open(img_file).convert('L')  # Grayscale
                
                # Resize if necessary
                if img.size != image_size:
                    img = img.resize(image_size, Image.Resampling.LANCZOS)
                
                # Convert to numpy array and normalize
                img_array = np.array(img, dtype=np.float32) / 255.0
                
                all_data.append(img_array)
                all_labels.append(class_idx)
                
            except Exception as e:
                print(f"      ❌ Failed to load {img_file}: {e}")
    
    if len(all_data) == 0:
        raise ValueError("No data loaded! Check dataset directory and file formats.")
    
    # Convert to numpy arrays
    data_array = np.array(all_data)  # Shape: (N, H, W)
    labels_array = np.array(all_labels)
    
    print(f"   Loaded {len(data_array)} samples")
    print(f"   Data shape: {data_array.shape}")
    print(f"   Label distribution: {np.bincount(labels_array)}")
    
    # Create tensors in different formats
    # Convolutional format: (N, C, H, W)
    conv_data = torch.tensor(data_array, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
    
    # Linear format: (N, H*W) - flattened
    linear_data = torch.tensor(data_array, dtype=torch.float32).view(len(data_array), -1)
    
    # Labels
    labels_tensor = torch.tensor(labels_array, dtype=torch.long)
    
    return conv_data, linear_data, labels_tensor

try:
    # Load the dataset
    class_names = dataset_info['label_names']
    image_size = dataset_info['image_size']
    
    conv_data, linear_data, labels = load_dataset_with_proper_shapes(
        test_dataset_dir, class_names, image_size
    )
    
    print(f"\\n✅ Data loaded successfully!")
    print(f"   Convolutional format: {conv_data.shape}")
    print(f"   Linear format: {linear_data.shape}")
    print(f"   Labels: {labels.shape}")
    print(f"   Value range: [{conv_data.min():.3f}, {conv_data.max():.3f}]")
    
    # Split into train/test
    split_idx = int(len(conv_data) * 0.7)
    
    conv_train, conv_test = conv_data[:split_idx], conv_data[split_idx:]
    linear_train, linear_test = linear_data[:split_idx], linear_data[split_idx:]
    labels_train, labels_test = labels[:split_idx], labels[split_idx:]
    
    print(f"\\n📊 Data splits:")
    print(f"   Train samples: {len(conv_train)}")
    print(f"   Test samples: {len(conv_test)}")
    
except Exception as e:
    print(f"❌ Data loading failed: {e}")
    import traceback
    traceback.print_exc()
    raise

print("\\n✅ Data preprocessing complete!")
print("-" * 40)

In [ ]:
# 5. DATALOADER CREATION AND VALIDATION
print("\\n5. DataLoader Creation and Validation...")
print("-" * 40)

def create_dataloaders(conv_train, linear_train, labels_train, batch_size=4):
    """Create DataLoaders for both convolutional and linear models."""
    
    dataloaders = {}
    
    # Convolutional DataLoader
    # For autoencoders: (input, target, labels) where input=target for reconstruction
    conv_dataset = TensorDataset(conv_train, conv_train, labels_train)
    conv_loader = DataLoader(conv_dataset, batch_size=batch_size, shuffle=True)
    dataloaders['convolutional'] = conv_loader
    
    # Linear DataLoader
    linear_dataset = TensorDataset(linear_train, linear_train, labels_train)
    linear_loader = DataLoader(linear_dataset, batch_size=batch_size, shuffle=True)
    dataloaders['linear'] = linear_loader
    
    return dataloaders

try:
    # Create DataLoaders
    batch_size = 4  # Small batch size for testing
    dataloaders = create_dataloaders(conv_train, linear_train, labels_train, batch_size)
    
    print(f"✅ DataLoaders created successfully!")
    print(f"   Batch size: {batch_size}")
    
    # Test DataLoaders
    for model_type, loader in dataloaders.items():
        print(f"\\n   Testing {model_type} DataLoader:")
        print(f"      Total batches: {len(loader)}")
        
        # Get first batch and inspect
        first_batch = next(iter(loader))
        x, y, labels_batch = first_batch
        
        print(f"      Batch input shape: {x.shape}")
        print(f"      Batch target shape: {y.shape}")
        print(f"      Batch labels shape: {labels_batch.shape}")
        print(f"      Input/target equal: {torch.equal(x, y)}")
        
        # Verify shapes match expected model inputs
        if model_type == 'convolutional':
            expected_shape = (batch_size, 1, *image_size)
        else:
            expected_shape = (batch_size, input_size)
        
        if x.shape == expected_shape or x.shape[0] <= batch_size:  # Last batch might be smaller
            print(f"      ✅ Shape matches expected: {expected_shape}")
        else:
            print(f"      ❌ Shape mismatch! Expected: {expected_shape}, Got: {x.shape}")

except Exception as e:
    print(f"❌ DataLoader creation failed: {e}")
    import traceback
    traceback.print_exc()
    raise

print("\\n✅ DataLoader creation complete!")
print("-" * 40)

In [ ]:
# 6. EXPERIMENT RUNNER TESTING WITH PROPER DEBUG
print("\\n6. ExperimentRunner Testing...")
print("-" * 40)

def test_experiment_runner(architecture_name, model_params, train_loader, test_data, test_labels):
    """Test ExperimentRunner with a specific architecture."""
    
    print(f"\\n🧪 Testing {architecture_name}...")
    
    try:
        # Create ExperimentRunner
        runner = ExperimentRunner(
            output_dir=f"test_output_{architecture_name}",
            random_seed=42
        )
        print(f"   ✅ ExperimentRunner created")
        
        # Create model
        model = create_autoencoder(architecture_name=architecture_name, **model_params)
        print(f"   ✅ Model created: {model.__class__.__name__}")
        
        # Test data compatibility
        sample_batch = next(iter(train_loader))
        sample_input = sample_batch[0][:1]  # First sample from batch
        
        print(f"   Sample input shape: {sample_input.shape}")
        
        # Test model forward pass
        model.eval()
        with torch.no_grad():
            encoded, decoded = model(sample_input)
            print(f"   Model test - Encoded: {encoded.shape}, Decoded: {decoded.shape}")
        
        # Run training (just 1 epoch for testing)
        print(f"   🚀 Starting training...")
        
        trained_model, history = runner.train_autoencoder(
            model=model,
            train_loader=train_loader,
            test_data=test_data,
            test_labels=test_labels,
            epochs=2,  # Very short for testing
            learning_rate=0.001,
            class_names=class_names,
            save_model=False,  # Don't save during testing
            experiment_name=f"test_{architecture_name}",
            visualization_interval=999999,  # Disable intermediate visualizations
            num_visualizations=1  # Only final visualization
        )
        
        print(f"   ✅ Training completed successfully!")
        print(f"      Final train loss: {history.get('final_train_loss', 'N/A')}")
        print(f"      Final test loss: {history.get('final_test_loss', 'N/A')}")
        print(f"      Training time: {history.get('training_time', 0):.2f}s")
        
        return True, trained_model, history
        
    except Exception as e:
        print(f"   ❌ Training failed: {e}")
        import traceback
        traceback.print_exc()
        return False, None, None

# Test each working architecture
successful_runs = {}

for arch_name in ['simple_linear', 'convolutional']:  # Test key architectures first
    if arch_name not in created_models:
        print(f"\\n⏭️  Skipping {arch_name} - not available")
        continue
    
    model_params = created_models[arch_name]['params']
    
    # Select appropriate dataloader and test data
    if 'input_size' in model_params:
        # Linear model
        train_loader = dataloaders['linear']
        test_data = linear_test
    else:
        # Convolutional model  
        train_loader = dataloaders['convolutional']
        test_data = conv_test
    
    success, trained_model, history = test_experiment_runner(
        arch_name, model_params, train_loader, test_data, labels_test
    )
    
    if success:
        successful_runs[arch_name] = {
            'model': trained_model,
            'history': history,
            'params': model_params
        }

print(f"\\n🎉 Experiment Runner Testing Complete!")
print(f"   Successful runs: {len(successful_runs)}")
print(f"   Working architectures: {list(successful_runs.keys())}")
print("-" * 40)

In [ ]:
# 7. COMPREHENSIVE WRAPPER CLASS IMPLEMENTATION
print("\\n7. Comprehensive Wrapper Implementation...")
print("-" * 40)

class AutoEncoderExperimentWrapper:
    """
    Comprehensive wrapper for autoencoder experimentation.
    Handles all the complexity of data loading, model creation, and training.
    """
    
    def __init__(self, output_dir="wrapper_experiments", random_seed=42):
        self.output_dir = output_dir
        self.random_seed = random_seed
        self.runner = ExperimentRunner(output_dir=output_dir, random_seed=random_seed)
        
        # Storage for experiment results
        self.experiment_results = {}
        self.dataset_info = None
        self.data_cache = {}
        
        print(f"✅ AutoEncoderExperimentWrapper initialized")
        print(f"   Output directory: {output_dir}")
        print(f"   Random seed: {random_seed}")
    
    def prepare_dataset(self, dataset_type='layered_geological', output_dir=None, 
                       num_samples_per_class=10, image_size=(32, 32), num_classes=3):
        """Generate and load dataset for experiments."""
        
        if output_dir is None:
            output_dir = f"wrapper_dataset_{dataset_type}"
        
        print(f"\\n📊 Preparing dataset...")
        print(f"   Type: {dataset_type}")
        print(f"   Output: {output_dir}")
        print(f"   Samples per class: {num_samples_per_class}")
        print(f"   Image size: {image_size}")
        print(f"   Classes: {num_classes}")
        
        # Generate dataset
        self.dataset_info = generate_dataset(
            dataset_type=dataset_type,
            output_dir=output_dir,
            num_samples_per_class=num_samples_per_class,
            image_size=image_size,
            num_classes=num_classes,
            random_seed=self.random_seed,
            save_examples=True
        )
        
        # Load data in multiple formats - FIXED: Use correct variable names
        conv_data, linear_data, labels = load_dataset_with_proper_shapes(
            output_dir, self.dataset_info['label_names'], self.dataset_info['image_size']
        )
        
        # Split train/test
        split_idx = int(len(conv_data) * 0.7)
        
        self.data_cache = {
            'conv_train': conv_data[:split_idx],
            'conv_test': conv_data[split_idx:],
            'linear_train': linear_data[:split_idx],
            'linear_test': linear_data[split_idx:],
            'labels_train': labels[:split_idx],
            'labels_test': labels[split_idx:],
            'class_names': self.dataset_info['label_names'],
            'image_size': self.dataset_info['image_size']
        }
        
        print(f"✅ Dataset prepared successfully!")
        print(f"   Total samples: {len(conv_data)}")
        print(f"   Train: {len(self.data_cache['conv_train'])}")
        print(f"   Test: {len(self.data_cache['conv_test'])}")
        
        return self.dataset_info
    
    def run_single_experiment(self, architecture_name, latent_dim=16, epochs=5, 
                            learning_rate=0.001, batch_size=4):
        """Run a single autoencoder experiment."""
        
        if not self.data_cache:
            raise ValueError("Dataset not prepared. Call prepare_dataset() first.")
        
        print(f"\\n🧪 Running experiment: {architecture_name}")
        print(f"   Latent dim: {latent_dim}")
        print(f"   Epochs: {epochs}")
        print(f"   Learning rate: {learning_rate}")
        print(f"   Batch size: {batch_size}")
        
        # Determine model parameters
        image_size = self.data_cache['image_size']
        
        if architecture_name in ['simple_linear', 'deeper_linear']:
            model_params = {
                'input_size': image_size[0] * image_size[1],
                'latent_dim': latent_dim
            }
            train_data = self.data_cache['linear_train']
            test_data = self.data_cache['linear_test']
        else:
            model_params = {
                'input_channels': 1,
                'latent_dim': latent_dim,
                'input_size': image_size  # Add input_size for convolutional models too
            }
            train_data = self.data_cache['conv_train']
            test_data = self.data_cache['conv_test']
        
        # Create model
        model = create_autoencoder(architecture_name=architecture_name, **model_params)
        
        # Create DataLoader
        train_dataset = TensorDataset(train_data, train_data, self.data_cache['labels_train'])
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        # Run training
        experiment_name = f"{architecture_name}_latent{latent_dim}_lr{learning_rate}"
        
        trained_model, history = self.runner.train_autoencoder(
            model=model,
            train_loader=train_loader,
            test_data=test_data,
            test_labels=self.data_cache['labels_test'],
            epochs=epochs,
            learning_rate=learning_rate,
            class_names=self.data_cache['class_names'],
            save_model=True,
            experiment_name=experiment_name
        )
        
        # Store results
        self.experiment_results[experiment_name] = {
            'architecture': architecture_name,
            'model': trained_model,
            'history': history,
            'parameters': {
                'latent_dim': latent_dim,
                'epochs': epochs,
                'learning_rate': learning_rate,
                'batch_size': batch_size
            }
        }
        
        print(f"✅ Experiment completed: {experiment_name}")
        print(f"   Final test loss: {history.get('final_test_loss', 'N/A'):.4f}")
        
        return experiment_name, history
    
    def run_systematic_experiments(self, architectures=None, latent_dims=None, epochs=5):
        """Run systematic experiments across multiple configurations."""
        
        if architectures is None:
            architectures = ['simple_linear', 'convolutional']
        
        if latent_dims is None:
            latent_dims = [8, 16]
        
        print(f"\\n🚀 Running systematic experiments...")
        print(f"   Architectures: {architectures}")
        print(f"   Latent dims: {latent_dims}")
        print(f"   Epochs: {epochs}")
        
        results_summary = []
        
        for arch in architectures:
            for latent_dim in latent_dims:
                try:
                    exp_name, history = self.run_single_experiment(
                        architecture_name=arch,
                        latent_dim=latent_dim,
                        epochs=epochs
                    )
                    
                    results_summary.append({
                        'experiment': exp_name,
                        'architecture': arch,
                        'latent_dim': latent_dim,
                        'final_test_loss': history.get('final_test_loss', float('inf')),
                        'training_time': history.get('training_time', 0),
                        'status': 'success'
                    })
                    
                except Exception as e:
                    print(f"   ❌ Failed {arch} with latent_dim {latent_dim}: {e}")
                    results_summary.append({
                        'experiment': f"{arch}_latent{latent_dim}_failed",
                        'architecture': arch,
                        'latent_dim': latent_dim,
                        'final_test_loss': float('inf'),
                        'training_time': 0,
                        'status': f'failed: {str(e)[:50]}'
                    })
        
        print(f"\\n📊 Systematic experiments completed!")
        print(f"   Total experiments: {len(results_summary)}")
        print(f"   Successful: {sum(1 for r in results_summary if r['status'] == 'success')}")
        
        return results_summary
    
    def get_experiment_summary(self):
        """Get summary of all experiments."""
        
        if not self.experiment_results:
            print("No experiments have been run yet.")
            return None
        
        print(f"\\n📋 Experiment Summary ({len(self.experiment_results)} experiments):")
        print("-" * 60)
        
        for exp_name, exp_data in self.experiment_results.items():
            history = exp_data['history']
            params = exp_data['parameters']
            
            print(f"\\n🧪 {exp_name}")
            print(f"   Architecture: {exp_data['architecture']}")
            print(f"   Parameters: Latent={params['latent_dim']}, LR={params['learning_rate']}, Epochs={params['epochs']}")
            print(f"   Final test loss: {history.get('final_test_loss', 'N/A'):.4f}")
            print(f"   Training time: {history.get('training_time', 0):.2f}s")
            
        return self.experiment_results

# Test the wrapper
print(f"\\n🎯 Testing Comprehensive Wrapper...")

wrapper = AutoEncoderExperimentWrapper(
    output_dir="comprehensive_wrapper_demo",
    random_seed=42
)

print("\\n✅ Wrapper created successfully!")
print("-" * 40)

In [ ]:
# 8. WRAPPER DEMONSTRATION - SINGLE EXPERIMENT
print("\\n8. Wrapper Demonstration - Single Experiment...")
print("=" * 60)

# Prepare dataset using wrapper
print("\\n📊 Step 1: Preparing dataset...")
dataset_info = wrapper.prepare_dataset(
    dataset_type='layered_geological',
    num_samples_per_class=6,  # Small for quick demo
    image_size=(32, 32),
    num_classes=3
)

print("\\n🧪 Step 2: Running single experiment...")
try:
    experiment_name, history = wrapper.run_single_experiment(
        architecture_name='simple_linear',
        latent_dim=8,
        epochs=3,  # Quick demo
        learning_rate=0.001,
        batch_size=3
    )
    
    print(f"\\n✅ Single experiment completed successfully!")
    print(f"   Experiment: {experiment_name}")
    print(f"   Final loss: {history['final_test_loss']:.4f}")
    print(f"   Time: {history['training_time']:.2f}s")
    
except Exception as e:
    print(f"❌ Single experiment failed: {e}")
    import traceback
    traceback.print_exc()

print("\\n" + "=" * 60)

In [ ]:
# 9. WRAPPER DEMONSTRATION - SYSTEMATIC EXPERIMENTS
print("\\n9. Wrapper Demonstration - Systematic Experiments...")
print("=" * 60)

print("\\n🚀 Running systematic experiments...")
print("   This will test multiple architectures and latent dimensions")

try:
    results_summary = wrapper.run_systematic_experiments(
        architectures=['simple_linear', 'convolutional'],
        latent_dims=[4, 8],  # Small dimensions for quick demo
        epochs=2  # Very short for demo
    )
    
    print(f"\\n📊 Systematic Experiments Results:")
    print("-" * 50)
    
    for result in results_summary:
        status_emoji = "✅" if result['status'] == 'success' else "❌"
        print(f"{status_emoji} {result['experiment']}")
        print(f"      Architecture: {result['architecture']}")
        print(f"      Latent dim: {result['latent_dim']}")
        print(f"      Test loss: {result['final_test_loss']:.4f}")
        print(f"      Time: {result['training_time']:.2f}s")
        print(f"      Status: {result['status']}")
        print()
    
    # Find best result
    successful_results = [r for r in results_summary if r['status'] == 'success']
    if successful_results:
        best_result = min(successful_results, key=lambda x: x['final_test_loss'])
        print(f"🏆 Best result: {best_result['experiment']}")
        print(f"   Loss: {best_result['final_test_loss']:.4f}")
    
except Exception as e:
    print(f"❌ Systematic experiments failed: {e}")
    import traceback
    traceback.print_exc()

print("\\n" + "=" * 60)

In [ ]:
# 10. FINAL SUMMARY AND VALIDATION
print("\\n10. Final Summary and Validation...")
print("=" * 60)

# Get comprehensive summary
print("\\n📋 Getting experiment summary...")
experiment_summary = wrapper.get_experiment_summary()

print("\\n🔍 Validation Checks:")
print("-" * 30)

# Check 1: Dataset preparation
if wrapper.dataset_info:
    print("✅ Dataset preparation: SUCCESS")
    print(f"   Classes: {wrapper.dataset_info['label_names']}")
    print(f"   Total samples: {wrapper.dataset_info['total_samples']}")
else:
    print("❌ Dataset preparation: FAILED")

# Check 2: Data caching
if wrapper.data_cache:
    print("✅ Data caching: SUCCESS")
    print(f"   Train samples: {len(wrapper.data_cache['conv_train'])}")
    print(f"   Test samples: {len(wrapper.data_cache['conv_test'])}")
else:
    print("❌ Data caching: FAILED")

# Check 3: Experiment execution
if wrapper.experiment_results:
    print("✅ Experiment execution: SUCCESS")
    print(f"   Completed experiments: {len(wrapper.experiment_results)}")
    
    # Show best performing experiment
    best_exp = None
    best_loss = float('inf')
    
    for exp_name, exp_data in wrapper.experiment_results.items():
        test_loss = exp_data['history'].get('final_test_loss', float('inf'))
        if test_loss < best_loss:
            best_loss = test_loss
            best_exp = exp_name
    
    if best_exp:
        print(f"   Best experiment: {best_exp}")
        print(f"   Best test loss: {best_loss:.4f}")
        
else:
    print("❌ Experiment execution: FAILED")

# Check 4: File outputs
output_dirs = [
    wrapper.output_dir,
    "comprehensive_wrapper_test",
    "comprehensive_wrapper_demo"
]

files_created = 0
for output_dir in output_dirs:
    if os.path.exists(output_dir):
        files = list(Path(output_dir).rglob("*"))
        files_created += len(files)

if files_created > 0:
    print(f"✅ File outputs: SUCCESS ({files_created} files created)")
else:
    print("❌ File outputs: NO FILES CREATED")

print(f"\\n🎉 COMPREHENSIVE WRAPPER DEMO COMPLETE!")
print("=" * 60)
print("\\n📝 Summary:")
print(f"   ✅ Environment setup and imports working")
print(f"   ✅ Dataset generation and loading working")
print(f"   ✅ Model architectures tested and validated")
print(f"   ✅ Data preprocessing pipeline working")
print(f"   ✅ ExperimentRunner integration working")
print(f"   ✅ Comprehensive wrapper class implemented")
print(f"   ✅ Single and systematic experiments working")
print("\\n🚀 The autoencoder wrapper is ready for full-scale experimentation!")
print("\\n💡 Next steps:")
print("   1. Use wrapper.prepare_dataset() to generate larger datasets")
print("   2. Use wrapper.run_systematic_experiments() for comprehensive analysis")
print("   3. Extend wrapper class for custom experiment configurations")
print("   4. Integrate with visualization and analysis tools")
print("\\n" + "=" * 60)